### MAIN THEME: 해파리 품종 분류모델 만들기
- data ; datas/Project Jellyfish 폴더의 모든 데이터
- 다중 분류
- 

In [257]:
# 모델 불러오기
import pandas as pd
import numpy as np

import PIL.Image as Image
import matplotlib.pyplot as plt

import time

import torch
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torchvision import models
from torchvision.datasets import ImageFolder


import os
from glob import glob
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score
import seaborn as sns
import cv2


In [258]:
#!pip install torchsummary


In [259]:
# 모든 하위 폴더의 이미지 확인하기
file_folder = '../../datas/Project Jellyfish/Train_Test_Valid/Train'

trainimagelist = []
for folder in os.listdir(file_folder):  # 폴더 내 파일의 이름을 리스트로 저장
    for file in os.listdir(os.path.join(file_folder, folder)):  # 폴더 내 파일의 이름을 리스트로 저장
        image_path = os.path.join(file_folder, folder, file)    # 이미지 경로
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # 이미지 흑백으로 불러오기
        img_np = img.squeeze()  # 이미지 출력
        trainimagelist.append(img_np)
        #plt.imshow(img_np, cmap='gray') 
        #plt.show()
        
## img.np에 append 하기

feature / target 분리, 정규화

In [260]:
def load_images_from_folder(folder_path):
    subdirs = [f for f in os.listdir(folder_path)]
    
    datalist=[]
    labellist = []
    
    for sub in subdirs:
        
        subdir = folder_path + sub +'/'
        imgfile = [f for f in os.listdir(subdir)]
        
        for f in imgfile:
            img_path = subdir + f
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(np.array(img),(150,150))
            img=img/255.
            datalist.append(img)
            labellist.append(sub)
    return np.array(datalist),np.array(labellist)

In [261]:
trainpath = '../../datas/Project Jellyfish/Train_Test_Valid/Train/'
testpath = '../../datas/Project Jellyfish/Train_Test_Valid/test/'
validpath = '../../datas/Project Jellyfish/Train_Test_Valid/valid/'
trainimage,trlable = load_images_from_folder(trainpath)
testimage,tstlable  = load_images_from_folder(testpath)
validimage,vllable = load_images_from_folder(validpath)

# 매칭?
# 데이터 양? -> 늘려보자

In [282]:
validimage

array([[[0.25490196, 0.25882353, 0.25882353, ..., 0.31372549,
         0.32156863, 0.32941176],
        [0.25490196, 0.25882353, 0.2627451 , ..., 0.30980392,
         0.3254902 , 0.3372549 ],
        [0.25490196, 0.2627451 , 0.26666667, ..., 0.30196078,
         0.30196078, 0.30588235],
        ...,
        [0.24313725, 0.21960784, 0.2       , ..., 0.49411765,
         0.48627451, 0.45882353],
        [0.22745098, 0.23137255, 0.21176471, ..., 0.56862745,
         0.54509804, 0.48627451],
        [0.21176471, 0.23921569, 0.22352941, ..., 0.47058824,
         0.5372549 , 0.54509804]],

       [[0.49803922, 0.50588235, 0.57647059, ..., 0.10588235,
         0.10588235, 0.10196078],
        [0.47058824, 0.49411765, 0.54901961, ..., 0.10980392,
         0.10588235, 0.10588235],
        [0.44705882, 0.47058824, 0.51764706, ..., 0.11372549,
         0.10980392, 0.10980392],
        ...,
        [0.0627451 , 0.0627451 , 0.0627451 , ..., 0.00392157,
         0.00392157, 0.00392157],
        [0.0

In [263]:
from sklearn.preprocessing import LabelEncoder

labels = ['barrel_jellyfish', 'blue_jellyfish', 'compass_jellyfish', 'lions_mane_jellyfish', 'mauve_stinger_jellyfish', 'Moon_jellyfish']
le = LabelEncoder()
le.fit(labels)
trlabel_transformed = le.transform(trlable)
tstlabel_transformed = le.transform(tstlable)
valabel_transformed = le.transform(vllable)
    

len(tstlabel_transformed), tstlable.shape

(40, (40,))

In [264]:
# dataset class 만들기
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset

class Jellyfish(Dataset):
    
    def __init__(self, images, label):
        super(Jellyfish, self).__init__()
        # Tensor로 변환
        #images = np.array(images)
        self.feature = torch.tensor(images, dtype=torch.float32)#.reshape(-1, 150*150)
        # 라벨 생성
        self.labels = torch.tensor(label, dtype=torch.long)
        
    def __len__(self):
        return len(self.feature)
    
    def __getitem__(self, idx):
        return self.feature[idx], self.labels[idx]

In [265]:
# 데이터셋 만들기


train = Jellyfish(trainimage.reshape(-1,150*150), trlabel_transformed)
test = Jellyfish(testimage.reshape(-1,150*150), tstlabel_transformed)
valid = Jellyfish(validimage.reshape(-1,150*150), valabel_transformed)

In [266]:
train.feature.shape, train.labels.shape


(torch.Size([900, 22500]), torch.Size([900]))

In [267]:
batch_size = 20
train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=batch_size, shuffle=False)

In [268]:
for f, l in train_loader:
    print (f.shape, l.shape)

torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 22500]) torch.Size([20])
torch.Size([20, 

In [277]:
import torch.nn as nn
import torch.optim as optim
import torchmetrics.functional as metrics
from torch.utils.data import DataLoader

## 다중분류모델
## 입력층 피처 수
## 출력층 피처 수: 6(0 ~ 5)

class JellyFishModel(nn.Module):
    
    #모델 구성요소 초기화
    def __init__(self, in_, out_):
        super(JellyFishModel, self).__init__()
        self.fc1 = nn.Linear(in_, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, out_)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
    def forward(self,x):
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x
    

In [278]:
#device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LOSS_FN = nn.CrossEntropyLoss()
EPOCHS = 100
LEARNING_RATE = 0.001

#feature
in_, out_ = 150*150, 6

print(in_,out_)
#instance
model = JellyFishModel(in_, out_).to(DEVICE)
l1_regularizer = nn.L1Loss()

OPTIMIZER = optim.Adam(model.parameters(),lr=LEARNING_RATE)

22500 6


In [279]:
# 학습진행함수

def training(epoch,loader, kind='train'):
    #학습(활성화)
    model.train()
    train_report = [[],[],[]]
    losslist, acclist = [],[]
    totalscores = [losslist, acclist]
    score = 0
    for index, (image, label) in enumerate(loader):
        
        image,label = image.to(DEVICE), label.to(DEVICE)
        #print(image.shape,label.shape)
        #학습
        pre_label = model(image)
        #print(pre_label.shape)
        
        loss = LOSS_FN(pre_label,label)
        losslist.append(loss)
        #최적화
        
        if kind=='train':
            #최적화
            OPTIMIZER.zero_grad()
            loss.backward()
            OPTIMIZER.step()
        
        acc = accuracy_score(pre_label.argmax(dim = 1),label.detach().numpy())
        f1 = f1_score(pre_label.argmax(dim = 1),label.detach().numpy(),average='macro')
        acclist.append(f1)
        
        if score<f1:
            score = f1
            torch.save(model,'Jellyfish.pth')
            
        
        # 학습 진행 상태 출력
        
        #print(f' [LOSS] {loss},[ACC] {acc}')
        
    print(f'========{kind}======== {ep}/{EPOCHS} [LOSS] {sum(losslist)/len(train_loader):.3f},[F1] {sum(acclist)/len(train_loader):.3f}')
        

In [280]:
for ep in range(EPOCHS):
    training(ep,train_loader,'train')
    training(ep,valid_loader,'valid')
    print()

========train======== 0/100 [LOSS] 1.908,[F1] 0.137
========valid======== 0/100 [LOSS] 0.088,[F1] 0.004
========train======== 1/100 [LOSS] 1.639,[F1] 0.229
========valid======== 1/100 [LOSS] 0.083,[F1] 0.005
========train======== 2/100 [LOSS] 1.604,[F1] 0.288
========valid======== 2/100 [LOSS] 0.089,[F1] 0.005
========train======== 3/100 [LOSS] 1.590,[F1] 0.285
========valid======== 3/100 [LOSS] 0.092,[F1] 0.006
========train======== 4/100 [LOSS] 1.530,[F1] 0.322
========valid======== 4/100 [LOSS] 0.088,[F1] 0.006
========train======== 5/100 [LOSS] 1.439,[F1] 0.345
========valid======== 5/100 [LOSS] 0.096,[F1] 0.003
========train======== 6/100 [LOSS] 1.424,[F1] 0.368
========valid======== 6/100 [LOSS] 0.091,[F1] 0.006
========train======== 7/100 [LOSS] 1.350,[F1] 0.400
========valid======== 7/100 [LOSS] 0.096,[F1] 0.005
========train======== 8/100 [LOSS] 1.324,[F1] 0.421
========valid======== 8/100 [LOSS] 0.097,[F1] 0.004
========train======== 9/100 [LOSS] 1.312,[F1] 0.434
========vali